In [1]:
from selenium import webdriver as wd
import chromedriver_binary
import random
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from pushbullet import PushBullet
from pywebio.input import *
from pywebio.output import *
from pywebio.session import *
import time

In [2]:
from selenium import webdriver as wd
wd = wd.Chrome()
wd.implicitly_wait(10)

In [3]:
#VISA LOGIC
def start_searching():
    global state
    while (True):
        try:
            random_wait_time = random.randrange(5.0, 10.0)
            time.sleep(random_wait_time)
            find_loop()
            if state == "No existen citas disponibles en la ubicación seleccionada.":
                print("Trying again in 5 minute")
                time.sleep(300)
                break
            if not Found_Date == "None": #if there is a Found Date go ahead
                print("Found a Date")
                send_push()
                wd.switch_to.new_window('tab')
                url = "https://www.youtube.com/watch?v=MKI2XwlzjtU"
                page = wd.get(url)
                wd.implicitly_wait(10)
                time.sleep(30)
                break
            else:
                print("Trying again in 1 min")
                time.sleep(60)  
                visa_page()
            
        
        except TypeError:
                print("Updating Page")

#Go to Visa Page
def visa_page():
    url = "https://ais.usvisa-info.com/es-mx/niv/schedule/46713278/appointment"
    page = wd.get(url)

#Fill in all login data
def log_in():
    
#First Ok button
    ok_button = wd.find_element("xpath",'/html/body/div[7]/div[3]/div/button')
    ok_button.click()

    random_wait_time = random.randrange(1.0, 3.0)
    time.sleep(random_wait_time)

#Fill username
    user_email = wd.find_element("xpath",'//*[@id="user_email"]')
    user_email.send_keys("dyllan.egana@hotmail.com")

    random_wait_time = random.randrange(1.0, 3.0)
    time.sleep(random_wait_time)

#Fill Password
    user_password = wd.find_element("xpath",'//*[@id="user_password"]')
    user_password.send_keys("Dematic400")

    random_wait_time = random.randrange(1.0, 3.0)
    time.sleep(random_wait_time)

#Acepta terminos y condiciones
    terminos_condiciones = wd.find_element("xpath",'//*[@id="sign_in_form"]/div[3]/label/div')
    terminos_condiciones.click()

    random_wait_time = random.randrange(1.0, 3.0)
    time.sleep(random_wait_time)

    #Sign In button
    sign_in = wd.find_element("xpath",'//*[@id="sign_in_form"]/p[1]/input')
    sign_in.click()

    random_wait_time = random.randrange(1.0, 3.0)
    time.sleep(random_wait_time)

#clicks next to see the following months
def next_page():
    next1 = wd.find_element("xpath",'//*[@id="ui-datepicker-div"]/div[2]/div/a/span')
    next1.click()

#Search for the first date available on the current showing months
def find_date():
    global month1
    global Appt
    global day1
    soup = BeautifulSoup(wd.page_source)
    items = soup.find("td", {"data-handler": "selectDay"})
    month1 = int(items["data-month"]) + 1
    day1 = int(items.text)
    Appt = "2023" + "-" + str(day1) + "-" + str(month1)
        
#Loop to find the first appointment cycling on all months
def find_loop():
    #Open Calendar first appointment
    global Found_Date
    global state
    try:
        state = "Opening Calendar"
        open_calendar1 = wd.find_element("xpath",'//*[@id="appointments_consulate_appointment_date"]')
        open_calendar1.click()
    except:
        state = "No existen citas disponibles en la ubicación seleccionada."
        print(state)
        pass
    if state == "Opening Calendar": 
        random_wait_time = random.randrange(1.0, 3.0)
        time.sleep(random_wait_time)
        while (True):
            try:
                find_date()
                if month1 >= 3 and month1 < 5: #and day1 < 15 Less than June 15
                    print(f'FOUND A DATE!! {Appt}')
                    sss = '[data-month="'+ str(month1 - 1)+ '"]'
                    T2 = wd.find_element(By.CSS_SELECTOR, sss)
                    Found_Date = Appt
                    T2.click()

                else:
                    print(f'Try again [{Appt}]')
                    Found_Date = "None"
                break
            except TypeError:
                next_page()
                pass

def send_push():
    Whatsapp_text = (f'FOUND A DATE!! {Appt}')

    # Get the access token from Pushbullet.com
    access_token = "o.4B93MRQwFTFpLKPUdrRqptH4s1i0JOjB"

    # Get the instance using access token
    pb = PushBullet(access_token)

    # Send the data by passing the main title
    # and text to be send
    push = pb.push_note(Whatsapp_text,"https://ais.usvisa-info.com/es-mx/niv/schedule/46713278/appointment")
    

In [4]:
Number_requests = 0
state = "Start"
while (True):
    try:
        if Number_requests == 4 or Number_requests == 8 or state == "This site can’t be reached, trying again in 10 min":
            from selenium import webdriver as wd
            wd = wd.Chrome()
            wd.implicitly_wait(10)
        visa_page()
        random_wait_time = random.randrange(5.0, 10.0)
        time.sleep(random_wait_time)
        if not state == "No existen citas disponibles en la ubicación seleccionada.":
            log_in()
        start_searching()
    except:
        soup = BeautifulSoup(wd.page_source)
        items = soup.find("span", {"jstcache": "9"}).text
        if items == 'This site can’t be reached':
            state = 'This site can’t be reached, trying again in 10 min'
            print(state)
            Number_requests = Number_requests + 1
            if Number_requests == 4 or Number_requests == 8:
                state = "Start"
                print("Restarting Browser, trying again in 20 min")
                wd.quit()
                time.sleep(1200)
            if Number_requests == 12:
                print("Done for the DAY")
                wd.quit()
                break
            wd.quit()
            time.sleep(600)
        pass


Try again [2023-24-2]
Trying again in 1 min
Try again [2023-24-2]
Trying again in 1 min
Try again [2023-24-2]
Trying again in 1 min
Try again [2023-24-2]
Trying again in 1 min
Try again [2023-24-2]
Trying again in 1 min
Try again [2023-24-2]
Trying again in 1 min
Try again [2023-24-2]
Trying again in 1 min
Try again [2023-24-2]
Trying again in 1 min
Try again [2023-24-2]
Trying again in 1 min
No existen citas disponibles en la ubicación seleccionada.
Trying again in 5 minute
This site can’t be reached, trying again in 10 min
This site can’t be reached, trying again in 10 min
This site can’t be reached, trying again in 10 min
This site can’t be reached, trying again in 10 min
Restarting Browser, trying again in 20 min
This site can’t be reached, trying again in 10 min
This site can’t be reached, trying again in 10 min
This site can’t be reached, trying again in 10 min
This site can’t be reached, trying again in 10 min
Restarting Browser, trying again in 20 min
This site can’t be reached